<a href="https://colab.research.google.com/github/kky-ai/tech-demo/blob/main/semantic_continuity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Semantic Continuity Model - Example usage


##### Download model

In [ ]:
# Downlaod the AQ model from the GDrive

!gdown 1_uMuYyGDyXw98SK43dJRQYLY2KtSz46m && unzip -u sc.zip

##### Install libs

In [8]:
!pip install transformers==4.18
#!pip install tensorflow==2.6.2
!pip install numpy

ERROR: Could not find a version that satisfies the requirement tensorflow==2.7 (from versions: 0.12.1, 1.0.0, 1.1.0, 1.2.0, 1.2.1, 1.3.0, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.9.0, 1.10.0, 1.10.1, 1.11.0, 1.12.0, 1.12.2, 1.12.3, 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2)
ERROR: No matching distribution found for tensorflow==2.7


##### Initialize the model

In [ ]:
from transformers import AutoTokenizer
from transformers.optimization_tf import WarmUp
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

class SCModel:
    """ Semantic Continuity Model """

    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
        self.pad_token_id = self.tokenizer.pad_token_id
        self.cls_token_id = self.tokenizer.cls_token_id
        self.sep_token_id = self.tokenizer.sep_token_id
        self.mask_token_id = self.tokenizer.mask_token_id

        self.model, self.embedder, self.dist_model = None, None, None
        self.load_models()

    def load_models(self):
        self.model = load_model('sc/model', custom_objects={'WarmUp': WarmUp, 'siamese_loss': None})
        self.embedder = self.model.get_layer('embedder')
        self.dist_model = self.model.get_layer('dist_model')
        print('Models loaded.')

    def vectorize_seq(self, seq, max_len):
        seq = self.tokenizer.tokenize(seq)
        input_ids = [self.cls_token_id, *self.tokenizer.convert_tokens_to_ids(seq), self.sep_token_id]
        input_ids = pad_sequences([input_ids], maxlen=max_len, dtype=np.int32, padding='post', value=self.pad_token_id)
        return input_ids
        
    def score(self, prompt, reaction):
        _, pemb = self.embedder.predict(self.vectorize_seq(prompt, max_len=32))
        remb, _ = self.embedder.predict(self.vectorize_seq(reaction, max_len=64))
        dist, prob = self.dist_model.predict([pemb, remb])
        return float(dist)
    

sc = SCModel()

##### Example usage

In [7]:
q = 'What did you have for lunch?'

following = (
    'For lunch we had some fish and chips.',
    'We did not eat at all.',
    'We visited the city center. Then we had fish and chips.',
    'We visited the city center. Then we had some lunch.',
    'We\'ve been to the North London Derby. Arsenal was fantastic.'
)

print(f'Q: {q}', end='\n\n')
for context in following:
    dist = sc.score(q, context)
    print(f'[{round(dist, 4)}] {context}')

Q: What did you have for lunch?

[0.0305] For lunch we had some fish and chips.
[0.0432] We did not eat at all.
[0.4193] We visited the city center. Then we had fish and chips.
[1.5441] We visited the city center. Then we had some lunch.
[1.832] We've been to the North London Derby. Arsenal was fantastic.
